In [1]:
!pip install kora -q

     |████████████████████████████████| 57 kB 2.5 MB/s 
     |████████████████████████████████| 1.6 MB 8.3 MB/s 


In [3]:
#using selenium and bs4 daraz
from bs4 import BeautifulSoup
import requests
import re
from kora.selenium import wd #header for selenium in google colab may need !pip install kora -q

search_name=str(input("Enter the name of the product *try 'television 4k'* -->"))
#search_name="television 4k"
search_product=search_name.replace(' ','%20') #remove spaces
print("Calculating total number of pages .......")

wd.get(f'https://www.daraz.com.np/catalog/?q={search_product}')
code=BeautifulSoup(wd.page_source,"html.parser")

total_number_of_pages=int(code.find(class_="pager--mb0Ws").find_all('a')[-2].text)

print(f"Total number of pages for {search_name} are ---->{total_number_of_pages}")
number_of_pages=int(input(f"Out of {total_number_of_pages} pages, how many pages do you want to scrap? "))

all_data=[]
iname=[]#for graph
iprice=[]#for graph
for page in range(1,number_of_pages+1):
  print(f"Calculating for page {page}.......")
  wd.get(f'https://www.daraz.com.np/catalog/?page={page}&q={search_product}')
  code=BeautifulSoup(wd.page_source,"html.parser")
  sn=1
  items = code.find_all(class_="title--wFj93")
  for item in items:
    each_item=[]
    each_item.append(page)
    each_item.append(sn)
    #item name
    item_name = str(item.text)
    iname.append(item_name)#for graph
    each_item.append(item_name)
    #price
    item_price = int(item.next_sibling.text.replace(",","").replace("Rs. ",""))
    iprice.append(item_price)#for graph
    each_item.append(item_price)
    #discount(originalprice, discount rate, free shiping)
    item_discount = item.next_sibling.next_sibling
    item_original_price = item_discount.find(class_="origPrice--AJxRs").text if item_discount.find(class_="origPrice--AJxRs") else "None"
    each_item.append(item_original_price)
    item_discount_rate = item_discount.find(class_="discount--HADrg").text if item_discount.find(class_="discount--HADrg") else "None"
    each_item.append(item_discount_rate)
    if(item_discount.find("i")):
      item_shipping="Free Shipping"
      each_item.append(item_shipping)
    else:
      item_shipping="No Free Shipping"
      each_item.append(item_shipping)
    # rating and review
    item_ratingreview=item.next_sibling.next_sibling.next_sibling
    if item_ratingreview.find_all("i"):
      item_ratings = item_ratingreview.find_all("i")
      item_rating=0
      for r in item_ratings:
        item_rating = item_rating + int(r["class"][1][5:-7])
      each_item.append(item_rating)
    else:
      item_rating = "None"
      each_item.append(item_rating)

    if item_ratingreview.find(class_="rating__review--ygkUy"):
      item_review = item_ratingreview.find("span").text[1:-1]
      each_item.append(item_review)
    else:
      item_review = "None"
      each_item.append(item_review)
    #link
    item_link = item.a["href"]
    each_item.append(item_link)
    all_data.append(each_item)
    sn=sn+1

print(all_data)


#loop over
#write CSV
import csv
header =["Page","SN","Item_name","Item_price","Item_Original_Price","Item_Discount_Rate","Shipping","Rating","Number_of_Review","Link"]
file = open('Daraz_Scrap.csv','w')
x = csv.writer(file)
x.writerow(header)
for i in all_data:  
  x.writerow(i)
file.close()
print()
print("-----------Graph------------")
#plot graph
import plotly.graph_objects as go
fig = go.Figure([go.Bar(x=iname, y=iprice)])
fig.show()
print()
print("----------Most and Least expensive-------------")
#most expensive and least price
max_price=max(iprice)
min_price=min(iprice)
for p in range(len(iprice)):
  if iprice[p]==max_price:
    print(f"In {number_of_pages} pages, Most expensive {search_name} is {iname[p]} and price is {max_price} {iprice[p]} NPR")
  if iprice[p]==min_price:
    print(f"In {number_of_pages} pages, Least expensive {search_name} is {iname[p]} and price is {min_price} {iprice[p]} NPR")


print("-----------------------")


Enter the name of the product *try 'television 4k'* -->refrigerator
Calculating total number of pages .......
Total number of pages for refrigerator are ---->64
Out of 64 pages, how many pages do you want to scrap? 2
Calculating for page 1.......
Calculating for page 2.......
[[1, 1, 'Palsonic Australia SRS100-DT 100 Ltrs Single Door Mini Refrigerator', 18100, 'Rs. 21,000', '-14%', 'Free Shipping', 46, '16', '//www.daraz.com.np/products/palsonic-australia-srs100-dt-100-ltrs-single-door-mini-refrigerator-i100216635-s1020469713.html?search=1'], [1, 2, 'Yasuda Refrigerator YGDC200RE', 30170, 'Rs. 33,520', '-10%', 'Free Shipping', 'None', 'None', '//www.daraz.com.np/products/yasuda-refrigerator-ygdc200re-i117806991-s1032299202.html?search=1'], [1, 3, 'SAMSUNG RT28A3022GS - 253 Ltr Frost Free Double Door Refrigerator (Dit) - Silver', 47999, 'Rs. 54,490', '-12%', 'Free Shipping', 48, '45', '//www.daraz.com.np/products/samsung-rt28a3022gs-253-ltr-frost-free-double-door-refrigerator-dit-silver


----------Most and Least expensive-------------
In 2 pages, Least expensive refrigerator is Mini Fridge with Mirror LED Lights - 4 Liter and price is 6500 6500 NPR
In 2 pages, Most expensive refrigerator is Frigoglass Showcase/Visi Cooler 1000 Ltrs (European Design And Engineering) and price is 195556 195556 NPR
-----------------------
